In [1]:
import os
import gc
import warnings
warnings.filterwarnings('ignore')
import random
import scipy as sp
import numpy as np
import pandas as pd
import joblib
import itertools
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from itertools import combinations
import tensorflow as tf
import lightgbm as lgbm

In [2]:
# ====================================================
# Read data
# ====================================================
def read_data():
    input_dir = ''
    train = pd.read_parquet(input_dir + 'train_df_final.parquet')
    test = pd.read_parquet(input_dir + 'test_df_final.parquet')
    return train, test

In [3]:
gc.collect()

0

In [14]:
# ====================================================
# Amex metric
# ====================================================
def amex_metric(y_true, y_pred):
    labels = np.transpose(np.array([y_true, y_pred]))
    labels = labels[labels[1].argsort()[::-1]]
    weights = np.where(labels[:,0]==0, 20, 1)
    cut_vals = labels[np.cumsum(weights) <= int(0.04 * np.sum(weights))]
    top_four = np.sum(cut_vals[:,0]) / np.sum(labels[:,0])
    gini = [0,0]
    for i in [1,0]:
        labels = np.transpose(np.array([y_true, y_pred]))
        labels = labels[labels[:, i].argsort()[::-1]]
        weight = np.where(labels[:,0]==0, 20, 1)
        weight_random = np.cumsum(weight / np.sum(weight))
        total_pos = np.sum(labels[:, 0] *  weight)
        cum_pos_found = np.cumsum(labels[:, 0] * weight)
        lorentz = cum_pos_found / total_pos
        gini[i] = np.sum((lorentz - weight_random) * weight)
    return 0.5 * (gini[1]/gini[0] + top_four)

In [5]:
gc.collect()

0

In [6]:
# ====================================================
# tf amex metric
# ====================================================
def tf_amex_metric(y_pred, y_true):
    #y_true = y_true.get_label()
    return 'amex_metric', amex_metric(y_true, y_pred), True

In [7]:
gc.collect()

0

In [22]:
# ====================================================
# Training
# ====================================================
def train_(train, test):
    # Label encode categorical features
    cat_features = [
        "B_30",
        "B_38",
        "D_114",
        "D_116",
        "D_117",
        "D_120",
        "D_126",
        "D_63",
        "D_64",
        "D_66",
        "D_68"
    ]
    print("Encoding categorical variables")
    cat_features = [f"{cf}_last" for cf in cat_features]
    for cat_col in cat_features:
        encoder = LabelEncoder()
        train[cat_col] = encoder.fit_transform(train[cat_col])
        test[cat_col] = encoder.transform(test[cat_col])
        
    gc.collect()
        
    # Round last float features to 2 decimal place
    print("Rounding float features")
    num_cols = list(train.dtypes[(train.dtypes == 'float32') | (train.dtypes == 'float64')].index)
    num_cols = [col for col in num_cols if 'last' in col]
    for col in num_cols:
        train[col + '_round2'] = train[col].round(2)
        test[col + '_round2'] = test[col].round(2)
    
    # Filling NaN
    print("printing NaN values: ")
    print(train.isna().sum())
    print(test.isna().sum())
    
    train = train.replace(np.nan, 0)
    test = test.replace(np.nan, 0)
    
    print("After the fillna")
    print(train.isna().sum())
    print(test.isna().sum())
        
    # Get feature list
    print("Getting feature list")
    features = [col for col in train.columns if col not in ['customer_ID', "target"]]
    
    gc.collect()
    
    
    print("KFOLD Algorithm")
    kfold = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 42)
    for fold, (trn_ind, val_ind) in enumerate(kfold.split(train, train["target"])):
        X_train, X_test = train[features].iloc[trn_ind], train[features].iloc[val_ind]
        y_train, y_test = train["target"].iloc[trn_ind], train["target"].iloc[val_ind]
        
    print("Creating model")
    
    params ={'objective': 'binary',
        'metric': "binary_logloss",
        'boosting': 'dart',
        'seed': 42,
        'num_leaves': 100,
        'learning_rate': 0.01,
        'feature_fraction': 0.20,
        'bagging_freq': 10,
        'bagging_fraction': 0.50,
        'n_jobs': -1,
        'lambda_l2': 2,
        'min_data_in_leaf': 40
        }
    
    lgb_train = lgbm.Dataset(X_train, y_train)
    lgb_test = lgbm.Dataset(X_test, y_test)
    
    model = lgbm.train(params = params,
                       train_set = lgb_train,
                       num_boost_round = 300,
                       valid_sets = [lgb_train, lgb_test],
                       early_stopping_rounds = 100,
                       verbose_eval = 500)
    gc.collect()
    
    
    
    return model, X_test, y_test, test, features

In [23]:
def evaluate():
    preds = model.predict(X_test)
    
    print("Calculating metric....")
    #preds = preds.squeeze(axis = 1)
    metric = tf_amex_metric(preds, y_test)
    print(metric)

In [24]:
# ====================================================
# Predict
# ====================================================
def predict(test):
    predictions = model.predict(test[features])
    
    sub_df = pd.DataFrame({"customer_ID": test["customer_ID"], "prediction":predictions})
    sub_df.isna().sum()
    sub_df.head()
    
    return sub_df, predictions

In [25]:
# ====================================================
# to_csv
# ====================================================
def to_csv(sub_df, name):
    sub_df.to_csv(name, index = False)

In [26]:
gc.collect()

822

In [27]:
train_df, test_df = read_data()

In [28]:
gc.collect()

0

In [29]:
model, X_test, y_test, test, features = train_(train_df, test_df)

Encoding categorical variables
Rounding float features
printing NaN values: 
customer_ID               0
P_2_mean               2434
P_2_std                7829
P_2_min                2434
P_2_max                2434
                      ...  
D_141_last_round2      2532
D_142_last_round2    373333
D_143_last_round2      2532
D_144_last_round2         0
D_145_last_round2      2532
Length: 1096, dtype: int64
customer_ID               0
P_2_mean               4175
P_2_std                9937
P_2_min                4175
P_2_max                4175
                      ...  
D_141_last_round2      4607
D_142_last_round2    752615
D_143_last_round2      4607
D_144_last_round2         0
D_145_last_round2      4607
Length: 1095, dtype: int64
After the fillna
customer_ID          0
P_2_mean             0
P_2_std              0
P_2_min              0
P_2_max              0
                    ..
D_141_last_round2    0
D_142_last_round2    0
D_143_last_round2    0
D_144_last_round2    0
D_145_

AttributeError: 'Booster' object has no attribute 'fit'

In [ ]:
evaluate()

In [17]:
sub_df, predictions = predict(test)

In [18]:
test["customer_ID"].shape

(924621,)

In [20]:
to_csv(sub_df, "LGBMClassifier(withALLDATA)(booster=dart)(train_api).csv")

In [2]:
gc.collect()

517